<a href="https://colab.research.google.com/github/wajnryt/JerusMLDeepLearning2019/blob/master/DataLoader_Youtube_bb_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import glob
basedir = '/content/gdrive/My Drive/videos_2/yt_bb_detection_train'

In [0]:
import pandas as pd
import os
from torch.utils.data.dataset import Dataset
import imageio
from PIL import Image

class InstanceDataset(Dataset):
    def __init__(self, basedir, transform=None, force=False):
        super().__init__()
        self.transform = transform
        cach_file = os.path.join(basedir, 'data.csv')
        if not force and os.path.exists(cach_file):
          self.data = pd.read_csv(cach_file)
          print('load from csv')
          return
        print('start glob')
        files = glob.glob(os.path.join(basedir ,'*','*','*.jpg'))
        print('finish glob')
        self.data = pd.DataFrame([self._split_file(f) for f in files], 
                            columns=['class_id', 'instance_id', 'file_path'])
        self.data.to_csv(cach_file)
        print('finish sort')
        
    def _split_file(self, f):
        parts = f.split(os.sep)[-3:-1]
        return parts[0], parts[1], f 

    def __getitem__(self, index):
      dat = self.data.iloc[index]
      img = Image.open(dat['file_path'])
      if self.transform:
        img = self.transform(img)
      return (img, dat['class_id'], dat['instance_id'])
              
    def __len__(self):
        return len(self.data)
      

In [139]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256,256)),
        #transforms.Pad(256),
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataset = InstanceDataset(basedir, data_transforms['train'], True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=3,
                                             shuffle=True, num_workers=4)

device = torch.device("cuda:0")


start glob
finish glob
finish sort


In [0]:
print(len(dataset.data['instance_id'].unique()), ' unique instances')
print(len(dataset.data['class_id'].unique()), ' classes')
print(len(dataset) , ' images')
import numpy as np

plt.figure(figsize=(15,15))
plt.hist(dataset.data['class_id'].values.astype(int), normed=True)
plt.title('images per class')
plt.xlabel('class id')

plt.figure(figsize=(15,15))
_, count = np.unique(dataset.data['instance_id'], return_counts=True)
plt.hist(count, normed=True)
plt.title('images per instance')
plt.xlabel('images per instance')

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
#example. 
# this is not how you're supposed to use dataset!
# you should axes it via torch DataLoader...
# never dirrectly call __getitem__, or access the members (dataset.data)

print(len(dataset))
im, class_id, instance_id = dataset.__getitem__(500)


#just for fun, lets find another one of this guy
thisguy = dataset.data[dataset.data['instance_id']==instance_id]
for ind, dat in thisguy.iterrows():
  plt.figure()
  plt.imshow(imageio.imread(dat['file_path']))
  plt.title(f'class id {dat["class_id"]}, instance {dat["instance_id"]}')


In [0]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes, instance= next(iter(dataloader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,3)
plt.figure(figsize=(15,15))
imshow(out)#, title=[class_names[x] for x in classes])